<a href="https://colab.research.google.com/github/panzershracker/Web-scraping/blob/master/HW_les_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests, re
from pprint import pprint
from collections import Counter as counter
from bs4 import BeautifulSoup as bs4

In [0]:
url = 'https://ru.wikipedia.org/wiki/'

target = 'дерево'.capitalize()

In [108]:
r = requests.get(url+target)

print(r.encoding)

soup = bs4(r.text, 'html.parser')

head = soup.h1.next.strip()

UTF-8


In [109]:
print(r.status_code)
print(f'Тема: {head}')

200
Тема: Дерево


In [0]:
# находим необходимые элементы для парсинга по тегам и переводим все в строковый формат
main_text = str(soup.find_all(['p', 'li', 'span', 'a']))

In [111]:
mask = '[а-яА-Я]{4,}'

# создаем обьект counter на на основе маски и текста из супа по ключевым тегам
words = counter(re.findall(mask, main_text))

print(len(words))

2260


In [112]:
# Наиболее частые слова на основной странице
for i in words.most_common(20):
  print(i)

('Редактировать', 79)
('раздел', 76)
('править', 76)
('язык', 60)
('Латинский', 56)
('деревьев', 55)
('деревья', 55)
('Дерево', 52)
('дерево', 39)
('Розовые', 36)
('растений', 35)
('страница', 32)
('дерева', 30)
('видов', 30)
('Сосновые', 30)
('Слива', 30)
('Кипарисовые', 27)
('отсутствует', 27)
('Википедия', 26)
('семейства', 25)


# !
#### Не знал как сделать дальше чтоб было красиво, поэтому применил фильтр startswith. Если будут рекомендации, как методами СУПА чисто вытащить ссылку (с уточнением класса, например) - буду рад услышать.

In [113]:
# Получаем из обьекта супа необходимые нам теги для парсинга. Используем css селектор

links_raw = soup.select('#mw-content-text > div > ul > li > a')

links = []

# используем фильтр .startswith для выбора целевых ссылок
for i in links_raw:
  if i.get('href').startswith('http'):
    links.append(i.get('href'))
    
pprint(links)

['http://transspot.ru/2013/05/31/pro-derevya-na-ulicax/',
 'http://ec-dejavu.ru/d/Derevo.html',
 'http://www.maleus.ru/index.php/news/129-tree',
 'http://www.na.fs.fed.us/spfo/pubs/silvics_manual/table_of_contents.htm',
 'http://hort.ifas.ufl.edu/woody/']


### Запилил весь парсинг по сторонним сайтам в одну комплексную функцию. Не красиво, моветон - зато быстро )

In [0]:
def links_pars(links, mask, count):
  # Функция парсинга сторонних сайтов из раздела "ссылки"
  # Аргументы: список ссылок, маска поиска, кол-во наиболее встречаемых слов
  
  # перебор по ссылкам из списка (из раздела "Ссылки")
  for link in links:
    
    r = requests.get(link)
    
    # меняем кодировку:
    r.encoding = r.apparent_encoding
    
    # создаем обьект супа
    soup = bs4(r.text, 'html.parser')
    
    # создаем счетчик(из модуля collections) на базе текста страницы
    words = counter(re.findall(mask, str(soup)))
    
    # Условие: если список слов пустой - меняем маску поиска regex на английскую
    if len(words) == 0:
      
      en_mask = '[a-zA-Z]{4,}'
      words = counter(re.findall(en_mask, str(soup)))
    
    # Выводим код ответа, адрес ресурса и наиболее встречаемые слова
    print(f' Status code for {link} is: {r.status_code}\n'
          f' Наиболее часто встречаемые слова в документе:\n {words.most_common(count)}\n'
          f'==================================================================================')

In [121]:
links_pars(links, mask, 5)

 Status code for http://transspot.ru/2013/05/31/pro-derevya-na-ulicax/ is: 200
 Наиболее часто встречаемые слова в документе:
 [('движения', 26), ('деревьев', 25), ('деревья', 19), ('городе', 17), ('Деревья', 16)]
 Status code for http://ec-dejavu.ru/d/Derevo.html is: 200
 Наиболее часто встречаемые слова в документе:
 [('Гваттари', 33), ('дерево', 26), ('дерева', 20), ('Делез', 16), ('мира', 14)]
 Status code for http://www.maleus.ru/index.php/news/129-tree is: 200
 Наиболее часто встречаемые слова в документе:
 [('окаменелости', 4), ('Сбор', 4), ('средств', 4), ('хостинг', 4), ('дизайн', 4)]
 Status code for http://www.na.fs.fed.us/spfo/pubs/silvics_manual/table_of_contents.htm is: 200
 Наиболее часто встречаемые слова в документе:
 [('class', 438), ('data', 358), ('naspf', 189), ('href', 149), ('https', 108)]
 Status code for http://hort.ifas.ufl.edu/woody/ is: 200
 Наиболее часто встречаемые слова в документе:
 [('href', 50), ('shtml', 42), ('class', 18), ('http', 12), ('style', 10

#!
#### Так как две последние ссылки ведут не на статьи а на полу пустые страницы - в них преобладают слова из кода, т.к. они составляют большинство встречающихся слов.